In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
import cv2
import time
import gzip

BLUE = [0, 0.4470, 0.7410]
RED = [0.8500, 0.3250, 0.0980]
YELLOW = [0.929, 0.6940, 0.1250]

In [15]:
# run_name = 'fully-connected_10'
path = './saved_policy/2-agent_WPM_fully-connected_stuck_1'

num_agents = 2
num_landmarks = num_agents
def load_data(path):
    # import pickle file from ./test_policy/test_trajectory.pkl
    # with open('./test_policy/test_'+run_name+'/test_trajectory.pkl', 'rb') as f:

    # load path+'/test_trajectory.pkl.gz' file with gzip
    with gzip.open(path+'/test_trajectory.pkl.gz', 'rb') as f:
        trajectories = pickle.load(f)
    # with open(path+'/test_trajectory.pkl.gz', 'rb') as f:
    #     trajectories = pickle.load(f)

    # row = []
    # row.append(episode_count+1)
    # row.append(episode_step)
    # for agent in env.world.agents:
    #     row.append(agent.state.p_pos[0])
    #     row.append(agent.state.p_pos[1])
    # for landmark in env.world.landmarks:
    #     row.append(landmark.state.p_pos[0])
    #     row.append(landmark.state.p_pos[1])
    # row.append(rew_n[0])
    # row.append(rew_n[1])
    # if arglist.num_agents == 3:
    #     row.append(rew_n[2])
    # episode_trajectory.append(row)

    data = []
    df = pd.DataFrame()

    for traj in trajectories:
        list_episode = []
        list_timestep = []
        list_agent1_x = []
        list_agent1_y = []
        list_agent2_x = []
        list_agent2_y = []
        list_agent3_x = []
        list_agent3_y = []
        list_land1_x = []
        list_land1_y = []
        list_land2_x = []
        list_land2_y = []
        list_land3_x = []
        list_land3_y = []
        list_reward_1 = []
        list_reward_2 = []
        list_reward_3 = []
        list_agent1_cum_reward = []
        list_agent2_cum_reward = []
        list_agent3_cum_reward = []
        list_agent1_dist = []
        list_agent2_dist = []
        list_agent3_dist = []
        cum_rew_1 = 0
        cum_rew_2 = 0
        cum_rew_3 = 0
        for row in traj:
                if num_agents == 3:
                    list_episode.append(row[0])
                    list_timestep.append(row[1])
                    list_agent1_x.append(row[2])
                    list_agent1_y.append(row[3])
                    list_agent2_x.append(row[4])
                    list_agent2_y.append(row[5])
                    list_agent3_x.append(row[6])
                    list_agent3_y.append(row[7])
                    list_agents_x = [row[2], row[4], row[6]]
                    list_agents_y = [row[3], row[5], row[7]]
                    list_land1_x.append(row[8])
                    list_land1_y.append(row[9])
                    list_land2_x.append(row[10])
                    list_land2_y.append(row[11])
                    list_land3_x.append(row[12])
                    list_land3_y.append(row[13])
                    list_landmarks_x = [row[8], row[10], row[12]]
                    list_landmarks_y = [row[9], row[11], row[13]]
                    list_reward_1.append(row[14])
                    list_reward_2.append(row[15])
                    list_reward_3.append(row[16])
                if num_agents == 2:
                    list_episode.append(row[0])
                    list_timestep.append(row[1])
                    list_agent1_x.append(row[2])
                    list_agent1_y.append(row[3])
                    list_agent2_x.append(row[4])
                    list_agent2_y.append(row[5])
                    list_agents_x = [row[2], row[4]]
                    list_agents_y = [row[3], row[5]]
                    list_land1_x.append(row[6])
                    list_land1_y.append(row[7])
                    list_land2_x.append(row[8])
                    list_land2_y.append(row[9])
                    list_landmarks_x = [row[6], row[8]]
                    list_landmarks_y = [row[7], row[9]]
                    list_reward_1.append(row[10])
                    list_reward_2.append(row[11])


                dists = []
                for agent_x, agent_y in zip(list_agents_x, list_agents_y):
                    dist = []
                    for land_x, land_y in zip(list_landmarks_x, list_landmarks_y):
                        dist.append(np.linalg.norm(np.array([agent_x, agent_y]) - np.array([land_x, land_y])))
                    dists.append(min(dist))
                if num_agents == 3:
                    cum_rew_1 += row[14]
                    cum_rew_2 += row[15]
                    cum_rew_3 += row[16]
                if num_agents == 2:
                    cum_rew_1 += row[10]
                    cum_rew_2 += row[11]
                list_agent1_cum_reward.append(cum_rew_1)
                list_agent2_cum_reward.append(cum_rew_2)
                if num_agents == 3:
                    list_agent3_cum_reward.append(cum_rew_3)
                list_agent1_dist.append(dists[0])
                list_agent2_dist.append(dists[1])
                if num_agents == 3:
                    list_agent3_dist.append(dists[2])


        trajectory_ = {'episode': list_episode,
                       'timestep': list_timestep,
                          'agent1_x': list_agent1_x,
                            'agent1_y': list_agent1_y,
                            'agent2_x': list_agent2_x,
                            'agent2_y': list_agent2_y,
                            'agent3_x': list_agent3_x,
                            'agent3_y': list_agent3_y,
                            'land1_x': list_land1_x,
                            'land1_y': list_land1_y,
                            'land2_x': list_land2_x,
                            'land2_y': list_land2_y,
                            'land3_x': list_land3_x,
                            'land3_y': list_land3_y,
                            'reward_1': list_reward_1,
                            'reward_2': list_reward_2,
                            'reward_3': list_reward_3,
                            'agent1_cum_reward': list_agent1_cum_reward,
                            'agent2_cum_reward': list_agent2_cum_reward,
                            'agent3_cum_reward': list_agent3_cum_reward,
                            'agent1_dist': list_agent1_dist,
                            'agent2_dist': list_agent2_dist,
                            'agent3_dist': list_agent3_dist
                       }
        if num_agents == 2:
            del trajectory_['agent3_x']
            del trajectory_['agent3_y']
            del trajectory_['land3_x']
            del trajectory_['land3_y']
            del trajectory_['reward_3']
            del trajectory_['agent3_cum_reward']
            del trajectory_['agent3_dist']
        data.append(trajectory_)
        
    # Initialize empty lists for aggregation
    aggregated_data = {key: [] for key in data[0].keys()}

    # Aggregate corresponding elements
    for d in data:
        for key in d:
            aggregated_data[key].extend(d[key])

    # Create a DataFrame
    df = pd.DataFrame(aggregated_data)

    return df, data

df, trajectories = load_data(path)


%matplotlib qt
import pygame
from IPython.display import clear_output

# drop the first element of the list for each list item in trajectory
# trajectory_ = [t[1:] for t in trajectory_]


pygame.init()

# Display settings
WIDTH, HEIGHT = 640, 480
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Multi-Agent and Landmarks Animation")

# Colors
BLUE = np.array([0, 0.4470*255, 0.7410*255])
RED = np.array([0.8500*255, 0.3250*255, 0.0980*255])
YELLOW = np.array([0.929*255, 0.6940*255, 0.1250*255])

WHITE = (255, 255, 255)
AGENTS_COLORS = [BLUE, RED, YELLOW]
LANDMARK_COLORS = [(50, 50, 50), (50, 50, 50), (50, 50, 50)]
BLACK = (20, 20, 20)

frames = []
# shift and scale the data (-1,1) to fit the screen size (0,500) and (0,500)
SCALE_FACTOR = 200  # given our dimensions and trajectory range
SCREEN_CENTER = (WIDTH // 2, HEIGHT // 2)

# Choose a font (using a default system font here)
font = pygame.font.SysFont("arial", 16)

def map_to_screen(pos):
    """Map a trajectory position to a screen position."""
    return int(pos[0] * SCALE_FACTOR + SCREEN_CENTER[0]), int(pos[1] * SCALE_FACTOR + SCREEN_CENTER[1])


def draw_entity(screen, x, y, color, size=.2*SCALE_FACTOR):
    pygame.draw.circle(screen, color, (int(x), int(y)), size)



def display_text(text, x, y, color=BLACK):
    """Render and display text on the screen at specified coordinates."""
    text_surface = font.render(text, True, color)
    screen.blit(text_surface, (x, y))

def capture_frame(screen):
    """Capture the current Pygame screen frame."""
    frame = pygame.Surface(screen.get_size())
    frame.blit(screen, (0, 0))
    frames.append(frame)


def save_frames_to_video(frames, filename, fps=30):
    """Save captured frames to a video file."""
    height, width = frames[0].get_size()
    size = (width, height)
    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'XVID'), fps, size)
    for frame in frames:
        # Convert Pygame surface to OpenCV format
        frame_rgb = pygame.surfarray.array3d(frame).transpose([1, 0, 2])
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        out.write(frame_bgr)

    out.release()

def main():
    episode = 1
    # Sample trajectory data
    trajectory_ = trajectories[episode]
    time.sleep(10)
    clock = pygame.time.Clock()
    running = True
    current_time = 0
    while running:
        screen.fill(WHITE)
        # Calculate the current positions of the agents and landmarks
        for i in range(69):

            cum_rew_1 = trajectory_['agent1_cum_reward'][i]
            cum_rew_2 = trajectory_['agent2_cum_reward'][i]
            if num_agents == 3:
                cum_rew_3 = trajectory_['agent3_cum_reward'][i]

            t0 = trajectory_['timestep'][i]
            t1 = trajectory_['timestep'][i+1]
            
            if t0 <= current_time < t1:
                # clear_output(wait=True)
                # print(current_time, t0, t1)
                alpha = (current_time - t0) / (t1 - t0)

                # Drawing agents
                for j in range(num_agents):  
                    x0, y0 = trajectory_['agent{}_x'.format(j+1)][i], trajectory_['agent{}_y'.format(j+1)][i]
                    x1, y1 = trajectory_['agent{}_x'.format(j+1)][i+1], trajectory_['agent{}_y'.format(j+1)][i+1]
                    # r0, r1 = data0[12 + j], data1[12 + j]
                    x = x0 * (1 - alpha) + x1 * alpha
                    y = y0 * (1 - alpha) + y1 * alpha
                    r = trajectory_['reward_{}'.format(j+1)][i]
                    screen_x, screen_y = map_to_screen((x, y))
                    draw_entity(screen, screen_x, screen_y, AGENTS_COLORS[j])
                    display_text(f"{r:.4f}", screen_x-25, screen_y-60)

                # Drawing landmarks
                for j in range(num_agents):
                    x0, y0 = trajectory_['land{}_x'.format(j+1)][i], trajectory_['land{}_y'.format(j+1)][i]
                    x1, y1 = trajectory_['land{}_x'.format(j+1)][i+1], trajectory_['land{}_y'.format(j+1)][i+1]
                    x = x0 * (1 - alpha) + x1 * alpha
                    y = y0 * (1 - alpha) + y1 * alpha
                    screen_x, screen_y = map_to_screen((x, y))
                    draw_entity(screen, screen_x, screen_y, LANDMARK_COLORS[j], size=.05*SCALE_FACTOR)

                display_text(f"agent 1 cumulative reward: {cum_rew_1:.2f}", 370, 10)
                display_text(f"agent 2 cumulative reward: {cum_rew_2:.2f}", 370, 30)
                if num_agents == 3:
                    display_text(f"agent 3 cumulative reward: {cum_rew_3:.2f}", 370, 50)
                display_text(f"episode: {episode}",10, 20)
                display_text(f"timestep: {trajectory_['timestep'][i]}", 120, 20)
            # else:
                # print('not in range', current_time, t0, t1)
        pygame.display.flip()   
        capture_frame(screen)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        
        clock.tick(100)
        # Update current time
        current_time += .1

        # Exit loop when trajectory ends
        if current_time >= 69:
            print('Now showing episode {}'.format(episode+1))
            episode += 1
            trajectory_ = trajectories[episode]
            current_time = 0
            # running = False

    pygame.quit()

if __name__ == "__main__":
    main()
    # save_frames_to_video(frames,'episode_{}.avi'.format(episode))


Now showing episode 2
Now showing episode 3
Now showing episode 4
Now showing episode 5
Now showing episode 6
Now showing episode 7
Now showing episode 8
Now showing episode 9
Now showing episode 10
Now showing episode 11
